### Noise Gaussian

In [5]:
from skimage.restoration import estimate_sigma
import sys
import numpy as np
import cupy as cp
import skimage
from Utils import (read_directories,save_pickle,save_results_to_xlsx)
from noisy_functions import (add_low_noise_gaussian,add_moderate_noise_gaussian, add_high_noise_gaussian)    
from nlm_functions import (compute_adaptive_q,select_best_h_using_adaptive_q)   


root_dir_output_high = f'/workspace/data/output/high_noisy'
dir_images_general = f'/workspace/data/input/general_images'

dir_out_nlm = f'{root_dir_output_high}/NLM'
dir_out_geonlm =  f'{root_dir_output_high}/GEONLM'
dir_out_bm3d =  f'{root_dir_output_high}/BM3D'
dir_out_results =  f'{root_dir_output_high}/results'

In [ ]:
from skimage.restoration import estimate_sigma
import sys
import numpy as np
import cupy as cp
import skimage
from Utils import (read_directories,save_pickle,save_results_to_xlsx)
from noisy_functions import (add_low_noise_gaussian,add_moderate_noise_gaussian, add_high_noise_gaussian)    
from nlm_functions import (compute_adaptive_q,select_best_h_using_adaptive_q)   


root_dir_output_high = f'/workspace/data/output/high_noisy'
dir_images_general = f'/workspace/data/input/general_images'

dir_out_nlm = f'{root_dir_output_high}/NLM'
dir_out_geonlm =  f'{root_dir_output_high}/GEONLM'
dir_out_bm3d =  f'{root_dir_output_high}/BM3D'
dir_out_results =  f'{root_dir_output_high}/results'


array_dir = read_directories(dir_images_general)

array_nln_high_filtereds = []

for file in array_dir:

    file_name = file
    img = skimage.io.imread(f'{dir_images_general}/{file_name}')
    if img.ndim == 4:  # (T, H, W, C) ou (T, H, W)
        img = img[0]

    # Se tiver alpha (RGBA), descarta
    if img.ndim == 3 and img.shape[-1] == 4:
        img = img[..., :3]

    # Converte RGB -> cinza se ainda tiver 3 canais
    if img.ndim == 3 and img.shape[-1] == 3:
        img = skimage.color.rgb2gray(img)  # retorna float em [0,1]

    # >>> NORMALIZAÇÃO CRÍTICA: leve tudo para [0,255] float32 <<<
    if img.dtype.kind == 'f':                     # tipicamente após rgb2gray
        img = (np.clip(img, 0, 1) * 255).astype(np.float32)
    else:
        img = img.astype(np.float32)

    m, n = img.shape

    # Ruído (σ em 0–255, sem alterar seus valores)
    ruidosa = add_high_noise_gaussian(img)

    # Clipa para [0,255]
    ruidosa = np.clip(ruidosa, 0, 255)

    # CPU/GPU
    img_noisse_gaussian_np = ruidosa.astype(np.float32)
    estimated_sigma_gaussian_np = estimate_sigma(img_noisse_gaussian_np)  # data_range implícito 255
    img_gpu_noisse_gaussian = cp.array(img_noisse_gaussian_np)

    h_nlm = compute_adaptive_q(estimated_sigma_gaussian_np)
    q_nlm_candidates = np.array([h_nlm + delta for delta in range(-100, 25, 1)])

    # IMPORTANTE: passe a imagem de referência também em [0,255]
    img_filtered_nlm, nlm_h, psnr_nlm, ssim_nlm, score_nlm = select_best_h_using_adaptive_q(
        image=img,                          # agora está em [0,255] float32
        image_gpu=img_gpu_noisse_gaussian,  # idem
        q_nlm_candidates=q_nlm_candidates,
        f=4, t=7,
        alpha=0.5,
    )

    dct = {
        'img_noisse_gaussian_np': img_noisse_gaussian_np,
        'img_filtered_nlm': img_filtered_nlm,
        'estimated_sigma_gaussian': estimated_sigma_gaussian_np,
        'nlm_h': nlm_h,
        'psnr_nlm': psnr_nlm,
        'ssim_nlm': ssim_nlm,
        'score_nlm': score_nlm,
        'file_name': file_name,
    }
    array_nln_high_filtereds.append(dct)

    skimage.io.imsave(f'{dir_out_nlm}/{file_name}',np.clip(img_filtered_nlm, 0, 255).astype(np.uint8))
save_pickle(array_nln_high_filtereds, dir_out_results, "array_nln_high_filtereds.pkl")

### GEONLM BM3D

In [7]:
from geonlm_functions import run_geonlm_pipeline
import time
from Utils import (save_results_to_xlsx, load_pickle)

vetor = load_pickle(dir_out_results, "array_nln_high_filtereds.pkl")
from bm3d import bm3d, BM3DProfile
from skimage.restoration import estimate_sigma
from skimage.color import rgb2gray
from skimage.filters import median
from skimage.morphology import disk

from skimage.metrics import peak_signal_noise_ratio, structural_similarity

array_gnlm_bm3d_high_filtereds = []
for array in vetor:

    img_noisse_gaussian_np =array['img_noisse_gaussian_np']
    
    nlm_h =array['nlm_h']
    file_name =array['file_name']
    psnr_nlm =array['psnr_nlm']
    ssim_nlm =array['ssim_nlm']
    score_nlm =array['score_nlm']
    estimated_sigma_gaussian = array['estimated_sigma_gaussian']

    print(f'filename: {file_name}')
    print(f'psnr_nlm: {psnr_nlm}')
    print(f'ssim_nlm: {ssim_nlm}')
    print(f'score_nlm: {score_nlm}')

    
    img = skimage.io.imread(f'{dir_images_general}/{file_name}')
    if img.ndim == 4:  # (T, H, W, C) ou (T, H, W)
        img = img[0]

    # 2) Se tiver alpha (RGBA), descarta
    if img.ndim == 3 and img.shape[-1] == 4:
        img = img[..., :3]

    # 3) Converte RGB -> cinza se ainda tiver 3 canais
    if img.ndim == 3 and img.shape[-1] == 3:
        img = skimage.color.rgb2gray(img)
    
    # Parâmetros
    f = 4
    t = 7
    nn = 10

    

    print("\n--- Executando Pipeline 512x512 com mult = 1.45 ---")
    mult = 1.45
    img_filtered_gnlm_145, h_gnlm_145, psnr_gnlm_145, ssim_gnlm_145, score_gnlm_145 = run_geonlm_pipeline(img, nlm_h, img_noisse_gaussian_np, f, t, mult, nn)   


    print("\n--- Executando Pipeline 512x512 com mult = 1.50 ---")
    mult = 1.50
    img_filtered_gnlm_150, h_gnlm_150, psnr_gnlm_150, ssim_gnlm_150, score_gnlm_150 = run_geonlm_pipeline(img, nlm_h, img_noisse_gaussian_np, f, t, mult, nn)        

    print("\n--- Executando Pipeline 512x512 com mult = 1.60 ---")
    mult = 1.60
    img_filtered_gnlm_160, h_gnlm_160, psnr_gnlm_160, ssim_gnlm_160, score_gnlm_160 = run_geonlm_pipeline(img, nlm_h, img_noisse_gaussian_np, f, t, mult, nn)   


    print("\n--- Executando Pipeline 512x512 com mult = 1.65 ---")
    mult = 1.65
    img_filtered_gnlm_165, h_gnlm_165, psnr_gnlm_165, ssim_gnlm_165, score_gnlm_165 = run_geonlm_pipeline(img, nlm_h, img_noisse_gaussian_np, f, t, mult, nn)       
      

    print("\n--- Executando Pipeline 512x512 com mult = 1.70 ---")
    mult = 1.70
    img_filtered_gnlm_170, h_gnlm_170, psnr_gnlm_170, ssim_gnlm_170, score_gnlm_170 = run_geonlm_pipeline(img, nlm_h, img_noisse_gaussian_np, f, t, mult, nn)       

    print("\n--- Executando Pipeline 512x512 com mult = 1.75 ---")
    mult = 1.75
    img_filtered_gnlm_175, h_gnlm_175, psnr_gnlm_175, ssim_gnlm_175, score_gnlm_175 = run_geonlm_pipeline(img, nlm_h, img_noisse_gaussian_np, f, t, mult, nn)  

    print("\n--- Executando Pipeline 512x512 com mult = 1.80 ---")
    mult = 1.80
    img_filtered_gnlm_180, h_gnlm_180, psnr_gnlm_180, ssim_gnlm_180, score_gnlm_180 = run_geonlm_pipeline(img, nlm_h, img_noisse_gaussian_np, f, t, mult, nn)  

    img_bm3d = img.copy()

    if len(img_bm3d.shape) > 2:
        img_bm3d = skimage.color.rgb2gray(img_bm3d)
        img_bm3d = 255 * img_bm3d
    # Remove frames extras, se existirem (e.g. GIF com shape (1, H, W) ou (N, H, W, 3))
    if img_bm3d.ndim == 4:
        img_bm3d = img_bm3d[0]
    elif img_bm3d.ndim == 3 and img_bm3d.shape[2] != 3:
        img_bm3d = np.squeeze(img_bm3d)

    # Converte para escala de cinza, se necessário
    if img_bm3d.ndim == 3 and img_bm3d.shape[2] == 3:
        img_bm3d_gray = rgb2gray(img_bm3d)  # retorna float64 em [0, 1]
    else:
        img_bm3d_gray = img_bm3d.astype(np.float32) / 255.0  # já está em cinza
     # Agora está garantidamente em escala de cinza float [0,1]
    img_bm3d_gray = np.clip(img_bm3d_gray * 255, 0, 255).astype(np.uint8)
    
     # 1. Normaliza a imagem ruidosa para intervalo [0, 1]
    ruidosa_normalizada = img_noisse_gaussian_np.astype(np.float32) / 255.0

    sigma_est = estimate_sigma(ruidosa_normalizada, channel_axis=None)

    inicio = time.time()
    # 3. Define o perfil padrão do BM3D
    perfil_bm3d = BM3DProfile()
    fim = time.time() 
    time_bm3d = fim - inicio

    # 4. Aplica o BM3D com os argumentos definidos
    denoised = bm3d(
        ruidosa_normalizada,
        sigma_psd=sigma_est,
        profile=perfil_bm3d
    )
    denoised_sq = np.squeeze(denoised)   

    skimage.io.imsave(f'{dir_out_bm3d}/{file_name}',  np.clip(denoised_sq * 255, 0, 255).astype(np.uint8))

    
    psnr_bm3d = peak_signal_noise_ratio(img_bm3d_gray, (denoised * 255).astype(np.uint8))
    ssim_bm3d = structural_similarity(img_bm3d_gray, (denoised * 255).astype(np.uint8))
    score_bm3d = 0.5 * psnr_bm3d + 0.5 * (ssim_bm3d * 100)    
   

    dict = {
        #'img_filtered_gnlm':img_filtered_gnlm,
        'nlm_h':nlm_h, 
        'h_gnlm_145':h_gnlm_145,
        'h_gnlm_150':h_gnlm_150,
        'h_gnlm_160':h_gnlm_160,
        'h_gnlm_165':h_gnlm_165,
        'h_gnlm_170':h_gnlm_170,
        'h_gnlm_175':h_gnlm_175,
        'h_gnlm_180':h_gnlm_180,
        'estimated_sigma_gaussian':estimated_sigma_gaussian,
      

        'ssim_nlm':ssim_nlm,
        'ssim_gnlm_145':ssim_gnlm_145,
        'ssim_gnlm_150':ssim_gnlm_150,       
        'ssim_gnlm_160':ssim_gnlm_160 ,
        'ssim_gnlm_165':ssim_gnlm_165 ,
        'ssim_gnlm_170':ssim_gnlm_170 ,
        'ssim_gnlm_175':ssim_gnlm_175,
        'ssim_gnlm_180':ssim_gnlm_180,
        'ssim_bm3d': ssim_bm3d,

        'psnr_nlm': psnr_nlm,
        'psnr_gnlm_145':psnr_gnlm_145,
        'psnr_gnlm_150':psnr_gnlm_150,       
        'psnr_gnlm_160':psnr_gnlm_160 ,
        'psnr_gnlm_165':psnr_gnlm_165 ,
        'psnr_gnlm_170':psnr_gnlm_170 ,
        'psnr_gnlm_175':psnr_gnlm_175,
        'psnr_gnlm_180':psnr_gnlm_180,
        'psnr_bm3d': psnr_bm3d,
        
        'score_nlm':score_nlm,   
        'score_gnlm_145':score_gnlm_145,
        'score_gnlm_150':score_gnlm_150,       
        'score_gnlm_160':score_gnlm_160 ,
        'score_gnlm_165':score_gnlm_165 ,
        'score_gnlm_170':score_gnlm_170 ,
        'score_gnlm_175':score_gnlm_175,
        'score_gnlm_180':score_gnlm_180,
        'score_bm3d': score_bm3d,        
       
        'file_name':file_name,

    }
    array_gnlm_bm3d_high_filtereds.append(dict)
save_pickle(array_gnlm_bm3d_high_filtereds, dir_out_results, "results_gaussian_gnlm_bm3d_high2.pkl")
save_results_to_xlsx(array_gnlm_bm3d_high_filtereds, dir_out_results, 'gnlm_bm3d_high_filtereds2.xlsx')

filename: 36.gif
psnr_nlm: 27.9319184645681
ssim_nlm: 0.8190475715195744
score_nlm: 54.91833780826277

--- Executando Pipeline 512x512 com mult = 1.45 ---

Executando GEONLM com h = 174.00 (base 120 * 1.45)
img_n.shape: (528, 528)
M: 520, N: 520
→ PSNR: 29.44, SSIM: 0.8478, Score: 57.11

--- Executando Pipeline 512x512 com mult = 1.50 ---

Executando GEONLM com h = 180.00 (base 120 * 1.5)
img_n.shape: (528, 528)
M: 520, N: 520
→ PSNR: 29.49, SSIM: 0.8487, Score: 57.18

--- Executando Pipeline 512x512 com mult = 1.60 ---

Executando GEONLM com h = 192.00 (base 120 * 1.6)
img_n.shape: (528, 528)
M: 520, N: 520
→ PSNR: 29.49, SSIM: 0.8484, Score: 57.17

--- Executando Pipeline 512x512 com mult = 1.65 ---

Executando GEONLM com h = 198.00 (base 120 * 1.65)
img_n.shape: (528, 528)
M: 520, N: 520
→ PSNR: 29.46, SSIM: 0.8474, Score: 57.10

--- Executando Pipeline 512x512 com mult = 1.70 ---

Executando GEONLM com h = 204.00 (base 120 * 1.7)
img_n.shape: (528, 528)
M: 520, N: 520
→ PSNR: 29.42

/workspace/src/gaussian_experiments/Utils.py:23: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(to_builtin)


PosixPath('/workspace/data/output/high_noisy/results/gnlm_bm3d_high_filtereds2.xlsx')

In [1]:
from skimage.restoration import estimate_sigma
import sys
import numpy as np
import cupy as cp
import skimage
from Utils import (read_directories,save_pickle,save_results_to_xlsx,load_pickle)
from noisy_functions import (add_low_noise_gaussian,add_moderate_noise_gaussian, add_high_noise_gaussian)    
from nlm_functions import (compute_adaptive_q,select_best_h_using_adaptive_q)   



vetor_result = load_pickle(f'/workspace/src/gaussian_experiments/results_gaussian_gnlm_bm3d_high.pkl', "object.pkl")

In [3]:
print(vetor_result)

[{'nlm_h': 120, 'h_gnlm': 186.0, 'estimated_sigma_gaussian': 15.768712112847876, 'ssim_nlm': 0.8190475715195744, 'ssim_gnlm': 0.8488403132683154, 'ssim_bm3d': 0.8718815877624065, 'psnr_nlm': 27.9319184645681, 'psnr_gnlm': 29.5056601991596, 'psnr_bm3d': 30.051876583048028, 'score_nlm': 54.91833780826277, 'score_gnlm': 57.19484576299557, 'score_bm3d': 58.62001767964434, 'time_geonlm': 209.84969687461853, 'time_bm3d': 3.0994415283203125e-06, 'file_name': '36.gif'}, {'nlm_h': 126, 'h_gnlm': 195.3, 'estimated_sigma_gaussian': 15.303111197559467, 'ssim_nlm': 0.860816668145439, 'ssim_gnlm': 0.8216276492305172, 'ssim_bm3d': 0.8904317036209292, 'psnr_nlm': 32.197989343282806, 'psnr_gnlm': 32.14104852005787, 'psnr_bm3d': 33.526365196946706, 'score_nlm': 59.139828078913354, 'score_gnlm': 57.151906721554795, 'score_bm3d': 61.28476777951981, 'time_geonlm': 204.0838429927826, 'time_bm3d': 1.9073486328125e-06, 'file_name': '39.gif'}, {'nlm_h': 120, 'h_gnlm': 186.0, 'estimated_sigma_gaussian': 15.2037

In [2]:
import pandas as pd
import numpy as np
#from caas_jupyter_tools import display_dataframe_to_user

# Rebuild the full dataset and save again (fixing the DataFrame conversion step)
data = vetor_result

df = pd.DataFrame(data)

# Apply elementwise coercion (correct method: applymap)
df = df.applymap(lambda x: float(x) if isinstance(x, (np.floating,)) else x)

# Column order
preferred_cols = [
    'file_name', 'estimated_sigma_gaussian',
    'nlm_h', 'h_gnlm',
    'psnr_nlm', 'psnr_gnlm', 'psnr_bm3d',
    'ssim_nlm', 'ssim_gnlm', 'ssim_bm3d',
    'score_nlm', 'score_gnlm', 'score_bm3d',
    'time_geonlm', 'time_bm3d'
]
cols = [c for c in preferred_cols if c in df.columns] + [c for c in df.columns if c not in preferred_cols]
df = df[cols]

xlsx_path = "gaussian_results_full.xlsx"

df.to_excel(xlsx_path, index=False, sheet_name="Results")




xlsx_path


/tmp/ipykernel_117149/856633759.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: float(x) if isinstance(x, (np.floating,)) else x)


ModuleNotFoundError: No module named 'openpyxl'

In [ ]:
from Utils import (save_results_to_xlsx, load_pickle)

from skimage.restoration import estimate_sigma
import sys
import numpy as np
import cupy as cp
import skimage
from Utils import (read_directories,save_pickle,save_results_to_xlsx)
from noisy_functions import (add_low_noise_gaussian,add_moderate_noise_gaussian, add_high_noise_gaussian)    
from nlm_functions import (compute_adaptive_q,select_best_h_using_adaptive_q)   


root_dir_output_high = f'/workspace/data/output/high_noisy'
dir_images_general = f'/workspace/data/input/general_images'

dir_out_nlm = f'{root_dir_output_high}/NLM'
dir_out_geonlm =  f'{root_dir_output_high}/GEONLM'
dir_out_bm3d =  f'{root_dir_output_high}/BM3D'
dir_out_results =  f'{root_dir_output_high}/results'


array_dir = read_directories(dir_images_general)

vetor = load_pickle(dir_out_results, "array_nln_high_filtereds.pkl")